 Experiment Data

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys

import data_process as dp

import os
if not os.path.exists('./Plots'):
    os.mkdir('./Plots')

from scipy.interpolate import UnivariateSpline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KernelDensity
import logit_analysis as logit

In [10]:
truncation_choices = [0.7, 0.8, 0.9]
plot_legend = []
for el in truncation_choices:
    plot_legend.append("Cap " + str(el))

df = pd.read_pickle('full_experiment_data.pkl')
print(df.shape[0],"original number of subjects")
for el in truncation_choices:
    print(df[df['probability_truncation'] == el].shape[0], "subjects in %.1lf cap group" % el)
print('')

#Filtering out too many quiz fails
df = df[df['quiz_failed'] < 2]

#Discarding subjects who kept loosing buttons for too long
experiment_total = dp.Decision_Process(df, drop_first_button=False)
suboptimal_subjects = np.array([True if max([(btn[:,1]-btn[:,0]).max() for btn in subj]) > 20 else False for subj in experiment_total.subjects_button_activity])
df = df[~suboptimal_subjects]
print(df.shape[0],"subjects remain after filtering")

experiment_total = dp.Decision_Process(df, drop_first_button=False)
experiment_split = []
for el in truncation_choices:
    truncation_split = df[df['probability_truncation'] == el]
    experiment_split.append(dp.Decision_Process(truncation_split))
    print(truncation_split.shape[0], "subjects in %.1lf cap group remain after filtering" % el)

3632 original number of subjects
1215 subjects in 0.7 cap group
1203 subjects in 0.8 cap group
1214 subjects in 0.9 cap group

3390 subjects remain after filtering
1114 subjects in 0.7 cap group remain after filtering
1125 subjects in 0.8 cap group remain after filtering
1151 subjects in 0.9 cap group remain after filtering


In [11]:
num_buttons_experiment = []
top_button_to_all_experiment = []
for i, el in enumerate(experiment_split):
    print("Truncation " + str(truncation_choices[i]) + ":")
    num_buttons, top_button_to_all = el.analyze_largest_contribution()
    num_buttons_experiment.append(num_buttons)
    top_button_to_all_experiment.append(top_button_to_all)

Truncation 0.7:
The average number of buttons explored is 10.710053859964093
The average reward received is 47.98204667863555
Out of 1114 subjects in total 567 subjects (or 50.90%) got the largest contribution from the last button
The average contribution of the button with the largest contribution is 58.24 %
Truncation 0.8:
The average number of buttons explored is 8.884444444444444
The average reward received is 56.949333333333335
Out of 1125 subjects in total 640 subjects (or 56.89%) got the largest contribution from the last button
The average contribution of the button with the largest contribution is 64.85 %
Truncation 0.9:
The average number of buttons explored is 7.665508253692441
The average reward received is 64.69852302345787
Out of 1151 subjects in total 762 subjects (or 66.20%) got the largest contribution from the last button
The average contribution of the button with the largest contribution is 68.99 %


In [12]:
# code

experiment_07 = experiment_split[0]

# print everything for 0.7 experiment

button_changes_less_than_50 = []
button_changes_more_than_50 = []



for data in enumerate(experiment_07.subjects_button_activity):
    print("Subject " + str(data[0]) + " has explored " + str(len(data[1])) + " buttons")
    num_diff_button_pressed_before = 0
    num_diff_button_pressed_after = 0
    total_presses = 0
    for i,button in enumerate(data[1]):
        # print("Button " + str(i) + " has " + str(button[-1,0]) + " wins and " + str(button[-1,1]) + " fails" + ", :" + str(button[-1,0] + button[-1,1]))
        # total number of presses
        total_presses += button[-1,0] + button[-1,1]
        if(total_presses < 50):
            num_diff_button_pressed_before += 1
        else:
            num_diff_button_pressed_after += 1


    button_changes_less_than_50.append(num_diff_button_pressed_before)
    button_changes_more_than_50.append(num_diff_button_pressed_after)   



# print button changes less than 50
print("Button changes less than 50")
print(button_changes_less_than_50)
print("Button changes more than 50")
print(button_changes_more_than_50)
    




    





Subject 0 has explored 9 buttons
Subject 1 has explored 6 buttons
Subject 2 has explored 1 buttons
Subject 3 has explored 6 buttons
Subject 4 has explored 2 buttons
Subject 5 has explored 5 buttons
Subject 6 has explored 5 buttons
Subject 7 has explored 8 buttons
Subject 8 has explored 5 buttons
Subject 9 has explored 6 buttons
Subject 10 has explored 13 buttons
Subject 11 has explored 7 buttons
Subject 12 has explored 4 buttons
Subject 13 has explored 8 buttons
Subject 14 has explored 10 buttons
Subject 15 has explored 1 buttons
Subject 16 has explored 1 buttons
Subject 17 has explored 8 buttons
Subject 18 has explored 1 buttons
Subject 19 has explored 12 buttons
Subject 20 has explored 5 buttons
Subject 21 has explored 2 buttons
Subject 22 has explored 7 buttons
Subject 23 has explored 6 buttons
Subject 24 has explored 11 buttons
Subject 25 has explored 10 buttons
Subject 26 has explored 6 buttons
Subject 27 has explored 3 buttons
Subject 28 has explored 11 buttons
Subject 29 has exp

Optimal data

In [13]:
! cd Simulations; make

g++  -o run  obj/run.o -lm
g++  -shared -o routines_lib.so  obj/routines_lib.o -lm


In [14]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

import os
import sys

import os
if not os.path.exists('Plots'):
    os.mkdir('Plots')

#Loading simulation routines
os.environ['SIMULATION_ROUTINES_PATH'] = os.getcwd() + '/Simulations/'
sys.path.append('Simulations')
import cpp_routines as sim
#Loading decision processing routines
import data_process as dp
#Loading logisic regression analysis routines
import logit_analysis as logit

In [16]:
run_time = 100
number_of_runs = 10000
button_prior = (1, 1)
policy_prior = (1, 1)

# button = sim.Button('Bernoulli', prior=(1, 1))
# button = sim.Button('Bernoulli', prior=button_prior)
button = sim.Button('Truncated', truncation=0.7)
# policy = sim.Policy('Bernoulli', prior=policy_prior)
policy = sim.Policy('Truncated', truncation=0.7)
# policy = sim.Policy('Difference', difference=4)

trial_record = sim.record_full_trials(run_time, number_of_runs, button, policy)

#Analysing decisions with data_process routines
processed_decisions = dp.Decision_Process(trial_record)

num_buttons_simulation, top_button_to_all_simulation = processed_decisions.analyze_largest_contribution()

The average number of buttons explored is 9.3515
The average reward received is 56.296
Out of 10000 subjects in total 9416 subjects (or 94.16%) got the largest contribution from the last button
The average contribution of the button with the largest contribution is 84.34 %


In [ ]:
# code

experiment_07 = processed_decisions

# print everything for 0.7 experiment

button_changes_less_than_50 = []
button_changes_more_than_50 = []



for data in enumerate(experiment_07.subjects_button_activity):
    print("Subject " + str(data[0]) + " has explored " + str(len(data[1])) + " buttons")
    num_diff_button_pressed_before = 0
    num_diff_button_pressed_after = 0
    total_presses = 0
    for i,button in enumerate(data[1]):
        # print("Button " + str(i) + " has " + str(button[-1,0]) + " wins and " + str(button[-1,1]) + " fails" + ", :" + str(button[-1,0] + button[-1,1]))
        # total number of presses
        total_presses += button[-1,0] + button[-1,1]
        if(total_presses < 50):
            num_diff_button_pressed_before += 1
        else:
            num_diff_button_pressed_after += 1


    button_changes_less_than_50.append(num_diff_button_pressed_before)
    button_changes_more_than_50.append(num_diff_button_pressed_after)   



# print button changes less than 50
print("Button changes less than 50")
print(button_changes_less_than_50)
print("Button changes more than 50")
print(button_changes_more_than_50)
    




    





Subject 0 has explored 19 buttons
Subject 1 has explored 4 buttons
Subject 2 has explored 19 buttons
Subject 3 has explored 24 buttons
Subject 4 has explored 5 buttons
Subject 5 has explored 6 buttons
Subject 6 has explored 3 buttons
Subject 7 has explored 22 buttons
Subject 8 has explored 6 buttons
Subject 9 has explored 18 buttons
Subject 10 has explored 16 buttons
Subject 11 has explored 6 buttons
Subject 12 has explored 14 buttons
Subject 13 has explored 2 buttons
Subject 14 has explored 5 buttons
Subject 15 has explored 15 buttons
Subject 16 has explored 10 buttons
Subject 17 has explored 1 buttons
Subject 18 has explored 15 buttons
Subject 19 has explored 14 buttons
Subject 20 has explored 8 buttons
Subject 21 has explored 6 buttons
Subject 22 has explored 14 buttons
Subject 23 has explored 2 buttons
Subject 24 has explored 11 buttons
Subject 25 has explored 4 buttons
Subject 26 has explored 7 buttons
Subject 27 has explored 6 buttons
Subject 28 has explored 17 buttons
Subject 29

In [ ]:
print("Number of buttons explored: ", num_buttons_simulation)



# get rewards for each subject
rewards_simulation = []
for data in processed_decisions.subjects_button_activity:
    rewards_simulation.append(sum([button[-1,0] for button in data]))

print("Rewards shape: ", np.array(x).shape)

print ("Average rewards: ", np.mean(rewards_simulation))



Number of buttons explored:  [20  1 15 ...  6  1  8]
Rewards shape:  (10000,)
Average rewards:  56.3293


In [ ]:
run_time = 100
number_of_runs = 10000
button_prior = (1, 1)
policy_prior = (1, 1)

# button = sim.Button('Bernoulli', prior=(1, 1))
# button = sim.Button('Bernoulli', prior=button_prior)
button = sim.Button('Truncated', truncation=0.8)
# policy = sim.Policy('Bernoulli', prior=policy_prior)
policy = sim.Policy('Truncated', truncation=0.8)
# policy = sim.Policy('Difference', difference=4)

trial_record = sim.record_full_trials(run_time, number_of_runs, button, policy)

#Analysing decisions with data_process routines
processed_decisions = dp.Decision_Process(trial_record)

num_buttons_simulation, top_button_to_all_simulation = processed_decisions.analyze_largest_contribution()
decisions_aggregated = dp.aggregate_wins_fails(processed_decisions.get_decisions(wins_more_fails=False))

The average number of buttons explored is 10.1131
The average reward received is 65.2358
Out of 10000 subjects in total 9520 subjects (or 95.20%) got the largest contribution from the last button
The average contribution of the button with the largest contribution is 84.24 %


In [ ]:
run_time = 100
number_of_runs = 10000
button_prior = (1, 1)
policy_prior = (1, 1)

button = sim.Button('Truncated', truncation=0.9)
policy = sim.Policy('Truncated', truncation=0.9)
# policy = sim.Policy('Difference', difference=4)

trial_record = sim.record_full_trials(run_time, number_of_runs, button, policy)

#Analysing decisions with data_process routines
processed_decisions = dp.Decision_Process(trial_record)

num_buttons_simulation, top_button_to_all_simulation = processed_decisions.analyze_largest_contribution()
decisions_aggregated = dp.aggregate_wins_fails(processed_decisions.get_decisions(wins_more_fails=False))

The average number of buttons explored is 8.9374
The average reward received is 75.1388
Out of 10000 subjects in total 9612 subjects (or 96.12%) got the largest contribution from the last button
The average contribution of the button with the largest contribution is 84.63 %


Processed decisions:  <data_process.Decision_Process object at 0x7f6c00928760>
Number of buttons simulation:  [15  1 11 ... 29  2  5]


TEST

In [ ]:
import numpy as np
from scipy import stats

# Assuming experiment_data and optimal_data are numpy arrays
# Replace these with actual data loading if necessary
experiment_data = np.array([...])
optimal_data = np.array([...])

cases = [0.9, 0.8, 0.7]

for case in cases:
    experiment_case = experiment_data[experiment_data == case]
    optimal_case = optimal_data[optimal_data == case]
    
    t_stat, p_val = stats.ttest_ind(experiment_case, optimal_case)
    
    print(f"For case {case}, the p-value is {p_val}")